In [1]:
from ultralytics import YOLO
import yaml
from pathlib import Path
import torch
import os

WARNING  Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\jmuri\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
from pathlib import Path
import os

print("Directorio actual de trabajo:", os.getcwd())

try:
    # Si es un script .py, usamos __file__
    script_path = Path(__file__).absolute()
    root_dir = script_path.parent.parent.parent
except NameError:
    # Si estamos en Jupyter/entorno interactivo
    script_path = None
    root_dir = Path.cwd().resolve()

print("Ubicación del script:", script_path if script_path else "No disponible en notebooks")
print("Directorio raíz calculado:", root_dir)
print("Carpeta data debería estar en:", root_dir / 'data')


Directorio actual de trabajo: c:\Users\jmuri\OneDrive\Documentos\Machine Learning\drone-traffic-monitor\src\training
Ubicación del script: No disponible en notebooks
Directorio raíz calculado: C:\Users\jmuri\OneDrive\Documentos\Machine Learning\drone-traffic-monitor\src\training
Carpeta data debería estar en: C:\Users\jmuri\OneDrive\Documentos\Machine Learning\drone-traffic-monitor\src\training\data


### __Class to train YOLO Model__

In [ ]:
class YOLOTrainer:
    def __init__(self, config_path='Config.yaml'):
        """
        Initialize YOLO trainer
        """
        if config_path is None:
            project_root = Path(__file__).parent.parent.parent
            config_path = project_root / 'config.yaml'

        self.config_path = config_path
        self.load_config()
        self.setup_training()

    def load_config(self):
        """
        Load model configuration from YAML file
        """
        try:
            with open(self.config_path, 'r') as file:
                self.config = yaml.safe_load(file)
                print("Configuration loaded successfully.")
        except Exception as e:
            print("Error loading configuration:", e)

            self.config = self.get_default_config()

    def get_default_config(self):
        """Default configuration for YOLO training."""
        return {
            'model': {
                'version': 'yolo11n',  # Usando YOLO11 más moderno
                'pretrained': True,
                'image_size': 640,
                'batch_size': 16,
                'epochs': 100,
                'patience': 50
            },
            'training': {
                'device': 'auto',
                'workers': 8,
                'learning_rate': 0.01,
                'momentum': 0.937,
                'weight_decay': 0.0005
            },
            'paths': {
                'data_yaml': 'data/data.yaml',
                'results': 'results/training'
            }
        }
    
    def setup_training(self):
        """
        Prepare the YOLO model for training
        """
        results_dir = Path(self.config['paths']['results'])
        results_dir.mkdir(parents=True, exist_ok=True)

        self.device = self.check_device()
        print("Training on device:", self.device)

        self.model = self.load_model()

    def check_device(self):
        """
        Check if CUDA is available and return the appropriate device
        """
        if torch.cuda.is_available():
            device = 'cuda'
            print("CUDA is available. Using GPU for training.")
        else:
            device = 'cpu'
            print("CUDA is not available. Using CPU for training.")
        
        return device
    
    def load_model(self):
        """
        Load YOLO model
        """
        model_version = self.config(['model']['version'])

        if self.config['model']['pretrained']:
            model = YOLO(f"{model_version}.pt")
            print(f"Loaded pretrained model: {model_version}")
        else:
            model = YOLO(f"{model_version}.yaml")
            print(f"Creating model from scratch: {model_version}")

        return model
    
    def create_data_yaml(self):
        """
        Create data.yaml file if it does not exist
        """
        data_yaml_path = Path(self.config['paths']['data_yaml'])
        
        if not data_yaml_path.exists():
            data_config = {
                'path': str(Path('data').absolute()),
                'train': 'VisDrone2019-DET-train/images',
                'val': 'VisDrone2019-DET-val/images',
                'test': 'VisDrone2019-DET-test-dev/images',
                'nc': 10,  # Número de clases
                'names': [
                    'pedestrian', 'people', 'bicycle', 'car', 'van',
                    'truck', 'tricycle', 'awning-tricycle', 'bus', 'motor'
                ]
            }
            
            data_yaml_path.parent.mkdir(parents=True, exist_ok=True)
            with open(data_yaml_path, 'w') as f:
                yaml.dump(data_config, f, default_flow_style=False)
            
            print(f"✓ Archivo data.yaml creado: {data_yaml_path}")
        else:
            print(f"✓ Archivo data.yaml encontrado: {data_yaml_path}")
        
        return data_yaml_path